In [ ]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv("../data/curated/property/property_addresses_to_lat_lng.csv")
stations_df = pd.read_csv("../data/curated/property/victoria_libraries.csv")

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
stations_df = stations_df.dropna(subset=['longitude', 'latitude'])




def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

station_features = stations_df.apply(lambda row: create_geojson_feature(row, 'station'), axis=1).tolist()
station_feature_collection = FeatureCollection(station_features)


houses_geojson_output_path = '../data/raw/external/API/3/property_3_library_geojson.geojson'
stations_geojson_output_path = '../data/raw/external/API/3/vic_library_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(stations_geojson_output_path, 'w') as f:
    geojson.dump(station_feature_collection, f)



print("Finished")


In [ ]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf6248dfd6833040c341ebb4797f7491012ac1'
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/3/property_3_library_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/3/vic_library_geojson.geojson', 'r') as f:
    stations_geojson = geojson.load(f)


def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
station_coordinates = extract_coordinates(stations_geojson)


house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
station_coords = [(lon, lat) for lat, lon, _ in station_coordinates]


def calculate_distance_matrix_in_batches(house_coords, station_coords, batch_size=20):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + station_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(station_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_station_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_station = station_coordinates[closest_station_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_station': closest_station,
                        'min_driving_distance_km': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)
    
    return all_shortest_distances


shortest_distances = calculate_distance_matrix_in_batches(house_coords, station_coords, batch_size=20)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/3/closest_library_distance_final_3.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"{output_path}")
